# Hessian and variance estimation for neural network

## Dataset

In [ ]:
def towdir(s):
    return (str('./datasets_book/'+s))

import deepglmlib.utils as utils
import numpy as np

In [ ]:
import importlib
importlib.reload(utils)

In [ ]:
import numpy as np

beta = np.array([-0.5,1.0,0.8,0.7,0.4,0.2]).reshape((6,1))
beta_true = beta

n = 30
x = np.random.uniform(0,1,n*5).reshape((n,5))
X = np.hstack([np.ones((n,1)), x])
e = np.random.randn(n).reshape((n,1))/5
y = X @ beta + e

x_train, x_test, y_train, y_test = utils.f_splitData(x,y,percentage=0.25)

x_train, x_test = utils.f_normalizeData(x_train,x_test)

In [ ]:
n_train, p_train = x_train.shape
n_test, p_test   = x_test.shape
X_train          = np.hstack([np.ones((n_train,1)), x_train])
X_test           = np.hstack([np.ones((n_test,1)), x_test])

In [ ]:
n_train, p_train, n_test, p_test 

## Variance estimation of regression parameters without  pytorch

### Standard-deviation estimation from **statsmodels**

In [ ]:
import statsmodels.api as stm
# n_train       = x_train.shape[0]
ols           = stm.OLS(y_train, X_train)
fit_ols_train = ols.fit()
olssumy       = fit_ols_train.summary()

print(olssumy.tables[1])

### Recall for some algebra on the variance of the coefficients


### Standard-deviation estimation from **numpy**

In [ ]:
def f_varthetahat(X,y,printed=False):
    n            = X.shape[0]
    p            = X.shape[1] #intercept not counting
    beta_hat     = np.linalg.inv(X.T @ X) @ X.T @ y
    y_hat        = X @ beta_hat
    residual     = y - y_hat
    sigma2_hat   = np.sum(residual**2) / (n - p)
    var_beta_hat = sigma2_hat * np.linalg.inv(X.T @ X)
    if printed:
        for p_ in range(p):
            standard_error = var_beta_hat[p_, p_] ** 0.5
            print(f"SE(beta_hat[{p_}]): {standard_error}")
    return beta_hat.ravel(), var_beta_hat, sigma2_hat

beta_hat_train, var_beta_hat_train, sigma2_hat_train = \
   f_varthetahat(X_train,y_train)

The covariance matrix estimate is equal to:

In [ ]:
Cov_betahat = var_beta_hat_train

print("Cov_betahat=\n", np.round(Cov_betahat,4))

## Variance parameters with pytorch: hessian computation

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

dataset_train = TensorDataset( torch.Tensor(x_train), torch.Tensor(y_train) )
dataset_test = TensorDataset( torch.Tensor(x_test), torch.Tensor(y_test) )

dl_train = DataLoader(dataset_train,shuffle=True,batch_size=10)
dl_test = DataLoader(dataset_test,shuffle=True,batch_size=10)

In [ ]:
print("cuda.is_available()     = ", torch.cuda.is_available())
print("cuda.get_device_name(0) = ",torch.cuda.get_device_name(0))

In [ ]:
device = torch.device("cuda:0" \
  if torch.cuda.is_available() else "cpu")

device

In [ ]:
import torch.nn as nn
import copy

px = x_train.shape[1]
nbmax_epoqs = 150
alpha_t     = 1e-3
debug_out   = 10

layers_regress = [ nn.Linear(px,1,bias=True) ]

model     =  utils.GNLMRegression("LinearRegression",
                                  copy.deepcopy(layers_regress))
loss      = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=alpha_t, momentum=0.0)
monitor   = utils.MyMonitorTest(model,loss,dl_train,dl_test,nbmax_epoqs,
                                debug_out,device=device)

loss_train_s,tmax,monistopc  = \
  utils.f_train_glmr(dl_train,model,optimizer,loss,
                     monitor,device=device,printed=1)

In [ ]:
loss_train_s = loss_train_s
t_train = range(len(loss_train_s))

loss_test_s = monitor.loss_test_s[monitor.loss_test_s>0]
t_test = monitor.step_test_s[monitor.loss_test_s>0].astype(int)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
utils.f_draw_s([ t_train, t_test ],
               [ loss_train_s/n_train,loss_test_s/n_test],
               ["b-", "r-"] ,"t",[ "loss train", "loss test"], " ", ax)

Thus, the vector of regression coefficient is written,

In [ ]:
#  utils.extract_weights_lin(model)
def extract_weights_lin(model,keybias="lin.bias",keyweight="lin.weight"):
    weight_ = bias_ = None
    for param_tensor in model.state_dict():
        if (param_tensor==keyweight):
            weight_= model.state_dict()[param_tensor]
        if (param_tensor==keybias):
            bias_= model.state_dict()[param_tensor]    
    return np.append(bias_,weight_)

In [ ]:
beta_hat_train2, var_beta_hat_train2, sigma2_hat_train2 = \
   f_varthetahat(X_train,y_train)

std_beta_hat_train2 = np.sqrt(np.diag(var_beta_hat_train2))

np.round(std_beta_hat_train2,3)

## Example with real data for classification

### Recall about the hessian and variance estimation for logistic regression

### Hessian and parameter variance for abalone dataset

#### Dataset

In [ ]:
import pandas as pd
abalone = pd.read_csv(towdir("./abalone_prep.csv"))

x       = abalone.drop(columns="rings")
y       = abalone["rings"].values - 1
y       = (y>np.median(y)).astype(int)

x.shape, y.shape

Let sample the train and set samples for the training.

In [ ]:
x_train, x_test, y_train, y_test = utils.f_splitData(x.values,y,percentage=0.333)
x_train, x_test                  = utils.f_normalizeData(x_train,x_test)

#### Results from the module statsmodels

In [ ]:
import statsmodels.api as stm
n_train       = x_train.shape[0]
X_train      = np.hstack([np.ones((n_train, 1)), x_train])
lgt           = stm.Logit(y_train, X_train)
fit_lgt_train = lgt.fit(maxiter=300)
lgtsumy       = fit_lgt_train.summary()

print(lgtsumy.tables[1])

#### Results from the module sklearn and variance with numpy 

In [ ]:
import numpy as np
from sklearn import linear_model
logit     = linear_model.LogisticRegression(penalty='none',fit_intercept=False,max_iter=300)
resLogit  = logit.fit(X_train, y_train)
predProbs = resLogit.predict_proba(X_train)
Omega     = np.diagflat(np.product(predProbs, axis=1))
cov_theta = np.linalg.inv(np.dot(np.dot(X_train.T, Omega), X_train))

betahat_skl = resLogit.coef_.ravel()
print("betahat_skl: ", np.round(betahat_skl,4))
print("stdhat_skl: ", np.round(np.sqrt(np.diag(cov_theta)),3))

#### Training with pytorch

In [ ]:
import torch

from torch.utils.data import TensorDataset, DataLoader

dataset_train     = TensorDataset( torch.Tensor(x_train), 
                                       torch.Tensor(y_train) )

dataset_test     = TensorDataset( torch.Tensor(x_test), 
                                       torch.Tensor(y_test) )

dl_train = DataLoader(dataset_train,shuffle=True,batch_size=100)
dl_test = DataLoader(dataset_test,shuffle=True,batch_size=100)

In [ ]:
device = torch.device("cuda:0" \
  if torch.cuda.is_available() else "cpu")

device

In [ ]:
import torch.nn as nn
import copy

px = x_train.shape[1]
nbmax_epoqs = 800
alpha_t     = 1e-2
debug_out   = 10

layers_regress = [ nn.Linear(px,1,bias=True) ]

model     =  utils.GNLMRegression("LogisticRegression",
                                  copy.deepcopy(layers_regress))

loss      = torch.nn.BCEWithLogitsLoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=alpha_t, momentum=0.0)
monitor   = utils.MyMonitorTest(model,loss,dl_train,dl_test,nbmax_epoqs,debug_out,device=device)

loss_train_s,tmax,monistopc  = utils.f_train_glmr(dl_train,model,optimizer,loss,monitor,device=device,
                                            transform_yb=utils.transform_yb,
                                            transform_yhatb=utils.transform_yhatb,
                                            printed=2)

In [ ]:
n_train, n_test = x_train.shape[0], x_test.shape[0]

In [ ]:
def extract_weights_lin(model,keybias="lin.bias",keyweight="lin.weight"):
    weight_ = bias_ = None
    for param_tensor in model.state_dict():
        if (param_tensor==keyweight):
            weight_= model.state_dict()[param_tensor]
        if (param_tensor==keybias):
            bias_= model.state_dict()[param_tensor]
    return np.append(bias_,weight_)

thetahat_torch = extract_weights_lin(model.cpu(),keybias="net.0.bias",keyweight="net.0.weight")
print( np.round(thetahat_torch,4) )

In [ ]:
logLik_torch=0
with torch.no_grad():
    for b, (Xb,yb) in enumerate(dl_train):
        yhatb = model(Xb)
        logLik_torch -= loss(yhatb.reshape(yb.shape), yb)

float(logLik_torch.detach().numpy())

In [ ]:
print(np.round(resLogit.coef_.ravel(),4))

In [ ]:
phat = np.exp(X_train @ resLogit.coef_.ravel())
phat = phat/(1+phat)
np.sum(y_train * np.log(phat) + (1-y_train)*np.log(1-phat))

#### Hessian computation with pytorch from second-order derivative

In [ ]:
import numpy as np
import torch, torchvision
from torch.autograd import Variable, grad
import torch.distributions as td
import math
from torch.optim import Adam
import scipy.stats

theta_train_ = torch.Tensor(thetahat_torch) 

In [ ]:
p_model = 0
for p in model.parameters():
    if len(p.shape)>1:
        p_model += p.shape[1]
    else:
        p_model += 1 #scalar (bias=intercept=wk0)
p_model

In [ ]:
I         = torch.zeros((p_model,p_model))
thessian  = torch.autograd.functional.hessian

optimizer = torch.optim.SGD(model.parameters(), lr=0.0, momentum=0.0)
theta_hat = theta_train_

t=0
for b,(Xb,yb) in enumerate(dl_train):
    print(".", end = '')
    Xb = torch.Tensor(np.hstack([np.ones((len(Xb), 1)), Xb]))
    #
    def log_lik_b(theta):
        p_b = torch.exp(Xb@theta)
        p_b = p_b/(1+p_b)
        return torch.log(p_b.T)@ yb +  torch.log(1-p_b.T) @ (1-yb)
    optimizer.zero_grad()
    I_b = -thessian(log_lik_b, theta_hat) / n_train
    I = I + I_b#.squeeze()

The hessian could be computed for the loss from the model, directly, this more advanced implementation is not given here and can be found as an extern module for pytorch, from repositories websites. This is mostly wanted for deep neural neworks where hidden layers are trained too and with their weighted added to the list of parameters.

The numpy version of the pytorch Tensor object which contains the hessian matrix is extracted.

In [ ]:
I = I.detach().numpy()

print()
print(np.round(I,3))

In [ ]:
np.asarray(lgtsumy.tables[1].data)[:,2][1:]

In [ ]:
np.round(np.sqrt(np.diag(np.linalg.inv(I))/n_train),3)

This is slightly equal to the standard-deviations from the estimation before with the dedicated python module. The difference comes from that the final solution for the regression coefficients were only nearby as the training is for a nonlinear function with different initial values and different inferential procedures. The first solution is a second-order procedure while the second one with pytorch is a first-order one which as for more carefull settings. This is the price to pay in order to get a more scalable algorithm without requiring the hessian at each step of the training, in order to avoid a non necessary costly numerical burden.

#### Hessian computation with pytorch from second-order derivative (bis)

Let revisit this python code with a more general setting with the model for computation of the predicted target variable. This is required for more advanced model with hidden layers for instance. First, let remember that the weights are structured as follows in pytorch.

In [ ]:
for p in model.parameters(): print(p.data, end= " ")

The weights are also available from the dictionary.

In [ ]:
mn = model.net[0]
print(mn.state_dict()['bias'], mn.state_dict()['weight'])

There is a more direct access to the weights which allows to update their values too.

In [ ]:
print(model.net[0].bias.data, model.net[0].weight.data)

In [ ]:
print(theta_hat[0], theta_hat[1:])

In [ ]:
def f_get_p_model(model):
    p_model = 0
    for p in model.parameters():
        if len(p.shape)>1:        # matrix          : array 2 dims
            p_model += p.shape[0] * p.shape[1]
        else:
            p_model += p.shape[0] # vector or scalar:  array 1 dim
    return p_model

In [ ]:
def f_varianceMatrixFromFullHessian_ForParameters_modelNN(model, loss, 
                                                          dataloader,n_train,
                                                          loss_yy_model = None,
                                                          device=None):
    if device is not None: model = model.to(device)
    model.eval()
    
    p_model = f_get_p_model(model)
    Imodel  = torch.zeros((p_model,p_model)) 
    if device is not None: Imodel = Imodel.to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0, momentum=0.0)
    optimizer.zero_grad()
    
    for b,(Xb,yb) in enumerate(dataloader):
        print(".", end = '')
        
        if device is not None: Xb=Xb.to(device)
            
        yhatb = model(Xb)
        if device is not None: yhatb = yhatb.to(device) # transformation ?????? <- linear & logit ! 
        
        #loss_b      = loss(yhatb, yb.reshape(yhatb.shape))
        # if loss_yy_model is None:
        
        if device is not None: yb = yb.to(device)
        #yb = transform_yb(yb, model.name,yhatb, device)
            
        loss_b = loss(yhatb, yb.reshape(yhatb.shape))
        # else:
        # loss_b = loss_yy_model(loss(yhatb, yb.reshape(yhatb.shape)),model)
               
        grad1rds_list = torch.autograd.grad(loss_b, model.parameters(), \
                                         create_graph=True, \
                                         retain_graph=True, \
                                         allow_unused=True)

        grad1rds_vec = torch.cat([g.view(-1) for g in grad1rds_list]) #.squeeze()
        
        grad2rds_list = []
        for grad1rd in  grad1rds_vec:
            grad2rds_1row = torch.autograd.grad(grad1rd, model.parameters(), \
                                          create_graph=True, \
                                          retain_graph=True, \
                                          allow_unused=True)

            grad2rds_1vect = torch.cat([g.view(-1) for g in grad2rds_1row]) #.squeeze()

            grad2rds_list.append( grad2rds_1vect )

        for k in range(p_model):
            Imodel[k,:] += grad2rds_list[k] / n_train
        
    return Imodel, p_model

###
Imodel_unroll, p_model = \
  f_varianceMatrixFromFullHessian_ForParameters_modelNN(model, loss, dl_train, n_train,
                                                        device=device)

Imodel_unroll = Imodel_unroll.detach().cpu().numpy()
Vtheta_hat = np.linalg.inv(Imodel_unroll) / n_train

print()
print(np.round(np.sqrt(np.diag(Vtheta_hat)),3))

#### Hessian computation with pytorch from first-order derivative

In [ ]:
def f_varianceMatrixFromGradients_ForParameters_modelNN(model, loss, dataloader, 
                                                        n_train, device=None):
    if device is not None: model = model.to(device)   
    p_model = f_get_p_model(model)
    Iapprox = torch.zeros((p_model,p_model))
    if device is not None: Iapprox = Iapprox.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.0, momentum=0.0)
    optimizer.zero_grad()

    for b,(Xb,yb) in enumerate(dataloader):
        print(".", end = '')
        for i in range(Xb.shape[0]):
            Xb_i = Xb[i,:]
            yb_i = yb[i].ravel()
            if device is not None: Xb_i=Xb_i.to(device)
            if device is not None: yb_i = yb_i.to(device)
            yhatb_i = model(Xb_i).ravel()
            loss_b = loss(yhatb_i, yb_i)
            optimizer.zero_grad()
            loss_b.backward()
            gradient_vect = []
            with torch.no_grad():
                for p in model.parameters():
                    gradient_vect.append(p.grad.view(-1))
                gradient_vect = torch.cat(gradient_vect)
                gradient_vect = gradient_vect.reshape((p_model,1))
                Iapprox = Iapprox + gradient_vect @ gradient_vect.T /n_train
    return Iapprox, p_model

In [ ]:
Iapprox, p_model = \
   f_varianceMatrixFromGradients_ForParameters_modelNN(model, loss, dl_train, 
                                                       n_train, device=device)

In [ ]:
Iapprox          = Iapprox.detach().cpu().numpy() 

In [ ]:
stdI1 = np.sqrt(np.diag(np.linalg.inv(Imodel_unroll))/n_train)
stdI2 = np.sqrt(np.diag(np.linalg.inv(Iapprox))/n_train)


stdI1 = np.roll(stdI1,1)
stdI2 = np.roll(stdI2,1)

print(np.round(stdI1,3))
print(np.round(stdI2,3))

In [ ]:
np.asarray(lgtsumy.tables[1].data)[:,2][1:]